# 4. baseline-3CH 
* 本bseline使用了smp中的U-Net，backbone为resnet101，仅使用RGB三通道
* 初次使用日期2019.11.8
* 未使用数据增强代码

In [1]:
# 所有图片使用RGB读入，没有做归一化
import os 
import cv2
import time 
import json 
import random 
import warnings

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from tqdm import tqdm
from timeit import default_timer as timer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, cohen_kappa_score, jaccard_score

import torch
import torchvision
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from torchvision import transforms as T
#from torchvision.models.segmentation import deeplabv3_resnet101

from imgaug import augmenters as iaa

from segmentation.utils import *

# from segmentation.unet_plus import SE_Res50UNet, SE_Res101UNet
import segmentation_models_pytorch as smp
from torchsummaryX import summary

GPU_ID = "0,1"
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_ID
multiGPU = len(GPU_ID.split(','))-1
print("使用%d块显卡"%(multiGPU+1))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用设备:%s"%(device))

使用2块显卡
使用设备:cuda


## 4.1 模型参数

In [4]:
ENCODER = 'vgg16'
SaveName = '05-baseline-3CH-UNet-seresnext50'
EPOCHES = 400
BS = 40
SIZE = [576,576, 3]
multiGPU = 0

if not os.path.exists('../weights/'+SaveName):
    os.makedirs('../weights/'+SaveName)
        
log = Logger()
log.open("../logs/{}.txt".format(SaveName),mode="a")

## 4.2 定义模型

In [5]:
# model = deeplabv3_resnet101(pretrained=False, progress=True, num_classes=16, aux_loss=None)
# #  导入模型与权重
model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights='imagenet', 
    classes=16, 
    activation='softmax',
)

summary(model.to(device),  torch.zeros(3, 3, 576,576).to(device))

                                                    Kernel Shape  \
Layer                                                              
0_encoder.features.Conv2d_0                        [3, 64, 3, 3]   
1_encoder.features.ReLU_1                                      -   
2_encoder.features.Conv2d_2                       [64, 64, 3, 3]   
3_encoder.features.ReLU_3                                      -   
4_encoder.features.MaxPool2d_4                                 -   
5_encoder.features.Conv2d_5                      [64, 128, 3, 3]   
6_encoder.features.ReLU_6                                      -   
7_encoder.features.Conv2d_7                     [128, 128, 3, 3]   
8_encoder.features.ReLU_8                                      -   
9_encoder.features.MaxPool2d_9                                 -   
10_encoder.features.Conv2d_10                   [128, 256, 3, 3]   
11_encoder.features.ReLU_11                                    -   
12_encoder.features.Conv2d_12                   

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder.features.Conv2d_0,"[3, 64, 3, 3]","[3, 64, 576, 576]",1792.0,5.733089e+08
1_encoder.features.ReLU_1,-,"[3, 64, 576, 576]",NaN,NaN
2_encoder.features.Conv2d_2,"[64, 64, 3, 3]","[3, 64, 576, 576]",36928.0,1.223059e+10
3_encoder.features.ReLU_3,-,"[3, 64, 576, 576]",NaN,NaN
4_encoder.features.MaxPool2d_4,-,"[3, 64, 288, 288]",NaN,NaN
5_encoder.features.Conv2d_5,"[64, 128, 3, 3]","[3, 128, 288, 288]",73856.0,6.115295e+09
6_encoder.features.ReLU_6,-,"[3, 128, 288, 288]",NaN,NaN
7_encoder.features.Conv2d_7,"[128, 128, 3, 3]","[3, 128, 288, 288]",147584.0,1.223059e+10
8_encoder.features.ReLU_8,-,"[3, 128, 288, 288]",NaN,NaN


## 4.3 随机数与数据读取、训练与验证函数

In [4]:
### 4.3.1 随机数设定
warnings.filterwarnings('ignore')
random.seed(13)
np.random.seed(13)
torch.manual_seed(13)
torch.cuda.manual_seed_all(13)
torch.backends.cudnn.benchmark = True

### 4.3.2 数据读取class
class Dataset(Dataset):
    """My Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images(numpy): image-npy(需要提前读入)
        masks (numpy): mask-npy(需要提前读入)
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """  
    def __init__(self, images, masks, size, augmentation=None, preprocessing=None, mode = 'train'):
        self.images = images
        self.masks = masks
        self.size = size
        self.augmentation = augmentation
        self.preprocessing = preprocessing
        self.mode = mode
        
    def __getitem__(self, i):
        # read data
        image = self.images[i]
        image = torch.from_numpy(image)
        image = image.permute(2, 0, 1).float()
        if self.mode == 'test':
            mask = '0'
        else:
            mask = self.masks[i]   
            mask = torch.from_numpy(mask)
            # mask = mask.permute(2, 0, 1).float()

            # apply augmentations
            if self.augmentation:
                sample = self.augmentation(image=image, mask=mask)
                image, mask = sample['image'], sample['mask']

            # apply preprocessing
            if self.preprocessing:
                sample = self.preprocessing(image=image, mask=mask)
                image, mask = sample['image'], sample['mask']

            # image = T.Compose([T.ToPILImage(),T.ToTensor()])(image) 
            # mask = mask.astype(np.uint8)
            # mask = T.Compose([T.ToPILImage(),T.ToTensor()])(mask)    
        
        return image, mask #.transpose(2,0,1)
        
    def __len__(self):
        return len(self.images)          

    def augumentor(self,image):
        # 数据增强使用了imgaug
        '''未完善，不可使用
        augment_img = iaa.Sequential([
            iaa.Fliplr(0.5),
            iaa.Flipud(0.5),
            iaa.SomeOf((0,4),[
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Affine(shear=(-16, 16)),
            ]),
            iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
            #iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
            ], random_order=True)
        '''
        image_aug = augment_img.augment_image(image)
        return image_aug

### 4.3.3 训练过程函数
def train(train_loader,model,criterion,optimizer,w,h):
    losses = AverageMeter()
    acc, jaccard, f1, f1_macro = AverageMeter(), AverageMeter(), AverageMeter(), AverageMeter()
    start = timer()
    model.train()
    for i,(image, mask) in enumerate(train_loader):
        image = image.to(device)
        b = image.size(0)
        mask = torch.from_numpy(np.array(mask)).long().to(device)
        # compute output
        output = model(image)
        loss = criterion(output, mask)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # print(loss.item())
        # total_loss += loss.item()
        losses.update(loss.item())
        # 计算评测指标并更新 metrics
        true = mask.cpu().data.numpy().reshape((b*w*h,1))
        pred = np.argmax(F.softmax(output).cpu().data.numpy(),axis=1).reshape((b*w*h,1))
        acc_score = accuracy_score(true, pred)
        #jaccard_batch = jaccard_score(true, pred, average='weighted')
        #f1_batch = f1_score(true, pred, average='weighted')
        #f1_batch_macro = f1_score(true, pred, average='macro')
        # kappa.update(kappa_score)
        acc.update(acc_score)
        #jaccard.update(jaccard_batch)
        #f1.update(f1_batch)
        #f1_macro.update(f1_batch_macro)
        
        print('\r',end='',flush=True)
        message = '%03d/%03d, %s\t===== loss:%0.5f, acc:%0.5f'%( #IoU:%0.5f, F1:%0.5f, F1-macro:%0.5f' % (
                  i, len(train_loader), time_to_str((timer() - start),'int'),
                  losses.avg, acc.avg)   #, jaccard.avg, f1.avg, f1_macro.avg ) 
        print(message, end='',flush=True)
        
    #log.write("\n")
    log.write(message)
    log.write("\n")
    return [losses.avg, acc.avg, jaccard.avg, f1.avg, f1_macro.avg]

# 4.3.4 验证过程函数
def evaluate(val_loader, model, criterion, w,h):
    losses = AverageMeter()
    kappa, acc, jaccard = AverageMeter(), AverageMeter(), AverageMeter()
    # switch mode for evaluation
    model.to(device)
    model.eval()
    with torch.no_grad():
        for i, (image,mask) in enumerate(val_loader):
            image_var = image.to(device)
            b = image_var.size(0) 
            mask = torch.from_numpy(np.array(mask)).long().to(device)
            # 计算loss
            output = model(image_var)
            loss = criterion(output, mask)
            losses.update(loss.item())
            # 计算评测指标并更新 metrics
            true = mask.cpu().data.numpy().reshape((b*w*h,1))
            pred = np.argmax(F.softmax(output).cpu().data.numpy(),axis=1).reshape((b*w*h,1))
            kappa_score = cohen_kappa_score(true, pred)
            acc_score = accuracy_score(true, pred)
            jaccard_batch = jaccard_score(true, pred, average='weighted')
            #f1_batch = f1_score(true, pred, average='weighted')
            #f1_batch_macro = f1_score(true, pred, average='macro')
            kappa.update(kappa_score)
            acc.update(acc_score)
            jaccard.update(jaccard_batch)
            #f1.update(f1_batch)
            #f1_macro.update(f1_batch_macro)
            print('\r',end='',flush=True)
            message = '\t\t=val= loss:%0.5f, kappa:%0.5f, acc:%0.5f, IoU:%0.5f'%( \
                            #, F1:%0.5f, F1-macro:%0.5f' % (\
                          losses.avg, kappa.avg, acc.avg, jaccard.avg) #, f1.avg, f1_macro.avg )
            print(message, end='',flush=True)
        #log.write("\n")
        log.write(message)
        log.write("\n")

    return [losses.avg, kappa.avg, acc.avg, jaccard.avg] #, f1.avg, f1_macro.avg]

## 4.4 读取数据集

In [5]:
train_npy = np.load("../data/train_1106.npy")
train_mask = np.load("../data/labels_1106.npy")
print(train_npy.shape, train_mask.shape)

val_npy = np.load("../data/test_1106.npy")
val_mask = np.load("../data/test_labels_1106.npy")
print(val_npy.shape, val_mask.shape)

(3960, 576, 576, 4) (3960, 576, 576)
(1560, 576, 576, 4) (1560, 576, 576)


In [4]:
# 本代码使用三通道
train_npy,val_npy = train_npy[:,:,:,1:4],val_npy[:,:,:,1:4]
print(train_npy.shape, train_mask.shape)

(3960, 576, 576, 3) (3960, 576, 576)


## 4.5 载入数据

In [6]:
train_dataset = Dataset(
    train_npy,
    train_mask, 
    size = SIZE,
    augmentation=None, #get_training_augmentation(), 
    preprocessing=None, #get_preprocessing(preprocessing_fn), #None
)

valid_dataset = Dataset(
    val_npy,
    val_mask,
    size = SIZE,
    augmentation=None, #get_training_augmentation(), 
    preprocessing=None, #get_preprocessing(preprocessing_fn), #None
)

train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=BS*2, shuffle=False, num_workers=1)

## 4.6 训练

In [9]:
lr = 0.001
# optimizer = optim.SGD(model.parameters(),lr = lr,momentum=0.9,weight_decay=1e-4)
optimizer = optim.Adam(model.parameters(), lr, weight_decay=1e-4, amsgrad=True)
#criterion = nn.NLLLoss2d().to(device)
criterion = nn.CrossEntropyLoss(reduction='sum', size_average=True).to(device)

start_epoch = 0
loss, kappa, acc, jaccard, f1, f1_macro = np.inf, 0, 0, 0, 0, 0
best_results = [loss, kappa, acc, jaccard] #, f1, f1_macro]
val_metrics = [loss, kappa, acc, jaccard] #, f1, f1_macro]

if torch.cuda.device_count() > 1:
    print('使用多显卡训练')
    model = nn.DataParallel(model)
model.to(device)

#scheduler = lr_scheduler.StepLR(optimizer,step_size=20,gamma=0.5) #, verbose=True)
#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=8, cooldown = 3,verbose=True)
#n_batches = int(len(train_loader.dataset) // train_loader.batch_size)
#scheduler = CosineAnnealingLR(optimizer, T_max=n_batches*2)

# 导入训练中断时的数据
# weight_name = "../weights/%s/kappa.pth.tar"%(SaveName)
# best_model = torch.load(weight_name)
# a = list(best_model['state_dict'])
# for i in range(len(a)):
#     a[i] = a[i].replace('module.','',1)
#     best_model['state_dict'][a[i]] = best_model['state_dict'].pop('module.'+a[i])
# del a
# model.load_state_dict(best_model['state_dict'])
# start_epoch = best_model["epoch"]
# best_results = [best_model["best_loss"], 
#                 best_model["best_kappa"], 
#                 best_model["best_acc"],
#                 best_model["best_jaccard"]]

### 4.4 Training
print("Start training...\t%d epoches"%(EPOCHES))
circle = 0
w,h = SIZE[0], SIZE[1]
for epoch in range(start_epoch,EPOCHES):
    now_time = time.strftime("%y-%m-%d %H:%M:%S", time.localtime())
    message = 'Epoch: %d\t======== 开始时间 %s ========\n'%(epoch, now_time)
    log.write(message,is_terminal=1)
    # scheduler.step(epoch)
    # train
    train_metrics = train(train_loader,model,criterion,optimizer,w,h)
    # val
    print('\n', end='',flush=True)
    val_metrics = evaluate(valid_loader,model,criterion,w,h)
    
    # check results
    is_best_loss = val_metrics[0] < best_results[0]
    is_best_kappa = val_metrics[1] > best_results[1]
    is_best_acc = val_metrics[2] > best_results[2]
    is_best_jaccard = val_metrics[3] > best_results[3]
    #is_best_f1, is_best_f1_macro = val_metrics[4] > best_results[4], val_metrics[5] > best_results[5]
    best_results[0] = min(val_metrics[0],best_results[0])
    for i in range(1,4):
        best_results[i] = max(val_metrics[i],best_results[i])
        
    # save model
    checkpoint = model.state_dict()
    save_checkpoint({"model_name":SaveName, "epoch":epoch + 1,
                     "best_loss":best_results[1], "best_kappa":best_results[0], "best_acc":best_results[2], 
                     "best_jaccard":best_results[3], 
                      # "best_f1":best_results[4],"best_f1_macro":best_results[5],
                     "state_dict":checkpoint, "optimizer":optimizer.state_dict()}, 
                    SaveName, epoch+1, 
                    is_best_loss,is_best_kappa, is_best_acc, is_best_jaccard)#, is_best_f1, is_best_f1_macro)

    print('\n', end='',flush=True)

    if is_best_loss:
        circle = 0
        print("==Get best LOSS in epoch %d: %0.8f "%(epoch,best_results[0]))
    else:
        circle +=1
    if is_best_kappa:
        circle = 0
        print("==Get best Kappa in epoch %d: %0.8f "%(epoch,best_results[1])) 
    if is_best_acc:
        circle = 0
        print("==Get best ACC in epoch %d: %0.8f"%(epoch,best_results[2]))
    if is_best_jaccard:
        circle = 0
        print("==Get best IoU in epoch %d: %0.8f"%(epoch,best_results[3]))
#     if is_best_f1:
#         circle = 0
#         print("==Get best F1 in epoch %d: %0.8f"%(epoch,best_results[4]))
#     if is_best_f1_macro:
#         circle = 0
#         print("==Get best F1-macro in epoch %d: %0.8f"%(epoch,best_results[5]))
    # new Lr    
    if circle >= 5:
        if lr >= 1e-8:
            lr *= 0.5
            circle=0
            optimizer = optim.Adam(model.parameters(), lr, weight_decay=1e-4, amsgrad=True)
            print("降低学习率至%0.10f,重新载入模型"%(lr))
            model.load_state_dict(torch.load( "../weights/%s/kappa.pth.tar"%(SaveName))['state_dict'])
        elif lr < 1e-12:
            break
        else:
            lr *= 0.8
            circle=0
            optimizer = optim.Adam(model.parameters(), lr, weight_decay=1e-4, amsgrad=True)
            print("降低学习率至%0.10f,不重新载入模型"%(lr))
            
    time.sleep(0.1)
    
log.write("\n LOSS:%0.8f\tKappa:%0.8f\tACC:%0.8f\tIoU:%0.8f"%(
            best_results[0], best_results[1], best_results[2],best_results[3]),is_terminal=1)
log.write("\n\n",is_terminal=1)

Start training...	400 epoches
Epoch: 16	======== 开始时间 19-11-08 18:22:07 ========
098/099, 259s	===== loss:0.25256, acc:0.91126
		=val= loss:0.95849, kappa:0.56846, acc:0.71671, IoU:0.58440
Epoch: 17	======== 开始时间 19-11-08 18:33:44 ========
098/099, 250s	===== loss:0.24018, acc:0.91622
		=val= loss:1.22709, kappa:0.48580, acc:0.64793, IoU:0.51532
Epoch: 18	======== 开始时间 19-11-08 18:44:49 ========
098/099, 248s	===== loss:0.20976, acc:0.92540
		=val= loss:0.93765, kappa:0.58704, acc:0.73419, IoU:0.60118
==Get best ACC in epoch 18: 0.73418539
==Get best IoU in epoch 18: 0.60118020
Epoch: 19	======== 开始时间 19-11-08 18:55:58 ========
098/099, 248s	===== loss:0.18223, acc:0.93438
		=val= loss:1.09026, kappa:0.52544, acc:0.68172, IoU:0.55433
Epoch: 20	======== 开始时间 19-11-08 19:07:05 ========
098/099, 247s	===== loss:0.21561, acc:0.92401
		=val= loss:1.73926, kappa:0.42373, acc:0.61511, IoU:0.48030
Epoch: 21	======== 开始时间 19-11-08 19:18:16 ========
098/099, 248s	===== loss:0.28799, acc:0.90000


098/099, 248s	===== loss:0.10132, acc:0.96402
		=val= loss:1.02101, kappa:0.60187, acc:0.74073, IoU:0.61499
Epoch: 64	======== 开始时间 19-11-09 03:21:55 ========
098/099, 248s	===== loss:0.09254, acc:0.96690
		=val= loss:1.06421, kappa:0.59660, acc:0.73822, IoU:0.60972
Epoch: 65	======== 开始时间 19-11-09 03:33:12 ========
098/099, 248s	===== loss:0.08868, acc:0.96826
		=val= loss:1.08964, kappa:0.59513, acc:0.73779, IoU:0.60920
Epoch: 66	======== 开始时间 19-11-09 03:44:29 ========
098/099, 248s	===== loss:0.08701, acc:0.96885
		=val= loss:1.11518, kappa:0.58802, acc:0.73154, IoU:0.60047
Epoch: 67	======== 开始时间 19-11-09 03:55:46 ========
098/099, 248s	===== loss:0.08214, acc:0.97033
		=val= loss:1.14955, kappa:0.58917, acc:0.73309, IoU:0.60386
降低学习率至0.0000625000,重新载入模型
Epoch: 68	======== 开始时间 19-11-09 04:06:56 ========
098/099, 246s	===== loss:0.17409, acc:0.93893
		=val= loss:0.87558, kappa:0.59789, acc:0.73841, IoU:0.60935
Epoch: 69	======== 开始时间 19-11-09 04:18:15 ========
098/099, 248s	===== 

098/099, 249s	===== loss:0.19648, acc:0.93072
		=val= loss:0.91251, kappa:0.58973, acc:0.73253, IoU:0.60172
Epoch: 114	======== 开始时间 19-11-09 12:46:32 ========
098/099, 249s	===== loss:0.19247, acc:0.93200
		=val= loss:0.91873, kappa:0.59008, acc:0.73181, IoU:0.60198
Epoch: 115	======== 开始时间 19-11-09 12:57:53 ========
098/099, 247s	===== loss:0.19296, acc:0.93188
		=val= loss:0.90699, kappa:0.59340, acc:0.73643, IoU:0.60610
Epoch: 116	======== 开始时间 19-11-09 13:09:07 ========
098/099, 249s	===== loss:0.19306, acc:0.93172
		=val= loss:0.91579, kappa:0.59050, acc:0.73319, IoU:0.60244
Epoch: 117	======== 开始时间 19-11-09 13:20:25 ========
098/099, 249s	===== loss:0.18962, acc:0.93287
		=val= loss:0.92604, kappa:0.58988, acc:0.73350, IoU:0.60264
降低学习率至0.0000000610,重新载入模型
Epoch: 118	======== 开始时间 19-11-09 13:31:45 ========
098/099, 248s	===== loss:0.19448, acc:0.93135
		=val= loss:0.92769, kappa:0.58906, acc:0.73326, IoU:0.60152
Epoch: 119	======== 开始时间 19-11-09 13:43:05 ========
098/099, 248s	

Traceback (most recent call last):
  File "/home/a430/anaconda3/envs/torch-1.1/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/a430/anaconda3/envs/torch-1.1/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/a430/anaconda3/envs/torch-1.1/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/a430/anaconda3/envs/torch-1.1/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

## 4.7 预测
### 4.7.1 预测参数

In [7]:
from segmentation.test import *
import skimage
import matplotlib.pyplot as plt
%matplotlib inline

SaveName = '00-baseline-3CH-UNet-resnet18'
metric = "kappa"
multiGPU = 0
NUM_TEST = 10     # 测试集大图的张数
BS = 40        # 训练集batch-size，测试集将×4
TestFileName = '../data/test_1106.npy'   # 测试集numpy位置
SIZE = [576,576, 3]

# 预测结果的输出文件夹
SAVENAME = ['20150212_L1A0000647768_4.npy', '20150217_L1A0000658637_2.npy', '20150902_L1A0001015649_2.npy',
            '20151203_L1A0001217916_1.npy', '20160225_L1A0001433318_3.npy', '20160327_L1A0001491417_3.npy',
            '20160421_L1A0001537716_2.npy', '20160510_L1A0001573999_2.npy', '20160816_L1A0001765570_2.npy',
            '20160827_L1A0001793003_2.npy']
SAVEPATH = os.path.join("../results", "{}_{}".format(SaveName, metric))
if not os.path.exists(SAVEPATH):
    os.makedirs(SAVEPATH)

### 4.7.2 载入预测模型

* **请运行4.5 定义模型部分代码**

In [9]:
weight_name =  "../weights/{}/{}.pth.tar".format(SaveName, metric)
if multiGPU:      #torch.cuda.device_count() > 1:     
    print('训练的模型为多卡，需要使用多卡预测或修改代码！')
    model = nn.DataParallel(model)
    
best_model = torch.load(weight_name)
# 轮次不同，进行预测。轮次相同不预测
model.load_state_dict(best_model["state_dict"])
model.to(device)
model.eval()

if metric != "checkpoint":
    best_metric = best_model['best_{}'.format(metric)]
    best_epoch = best_model["epoch"]
    print("使用模型为 %s == %s : %0.5f in %d epoch"%(weight_name, metric, best_metric, best_epoch))
else:
    print("使用模型为 %s/%s "%(SaveName, metric))

使用模型为 00-baseline-3CH-UNet-resnet18/kappa : 0.85141 in 16 epoch


### 4.7.3 读入预测文件与模型预测 

In [10]:
test_file = np.load(TestFileName)[:,:,:,1:4]   # 3CH代码需要变为三通道
num_test_each = test_file.shape[0]//NUM_TEST

for i in range(NUM_TEST):
    test_npy = test_file[num_test_each*i : num_test_each*(i+1)+1]
    test_dataset = Dataset(test_npy, np.array([[0]*len(test_npy)]), 
                           size = SIZE, 
                           augmentation=None, 
                           #get_training_augmentation(), 
                           preprocessing=None, 
                           #get_preprocessing(preprocessing_fn)
                           mode = 'test'
                          )
    test_loader = DataLoader(test_dataset, batch_size=BS*2, shuffle=False, num_workers=1)
    # 模型预测
    result = []
    for image,_ in test_loader:
        with torch.no_grad():
            image_var = image.to(device)
            # print(image_var.size())
            y_pred = model(image_var)    # 预测
            label = F.softmax(y_pred).cpu().data.numpy()
            # label形如(BS,16,576,576), 其中16为类别数,即置信度
            result.extend(label)
            
    result = np.array(result).transpose([0,2,3,1]) # 将list变为numpy并把置信度放在最后一个维度
    one_hot_result = get_large(result, unit_size=576, step=288, width=3600, height=3400)
    last = vis_label(one_hot_result)
    #plt.rcParams['figure.dpi'] = 120
    #plt.figure()
    #plt.imshow(last), plt.axis('off')
    print("Saving to {}/{}".format(SAVEPATH,SAVENAME[i]))    
    OutName = os.path.join(SAVEPATH, os.path.splitext(SAVENAME[i])[0]+'_test.tif')
    last = last.astype(np.uint8)    
    skimage.external.tifffile.imsave(OutName, last)

100%|██████████| 2/2 [00:12<00:00,  6.40s/it]


Saving to ../results/00-baseline-3CH-UNet-resnet18_kappa/20150212_L1A0000647768_4.npy


100%|██████████| 2/2 [00:03<00:00,  1.75s/it]


Saving to ../results/00-baseline-3CH-UNet-resnet18_kappa/20150217_L1A0000658637_2.npy


100%|██████████| 2/2 [00:03<00:00,  1.75s/it]


...

Saving to ../results/00-baseline-3CH-UNet-resnet18_kappa/20160510_L1A0001573999_2.npy


100%|██████████| 2/2 [00:03<00:00,  1.81s/it]


Saving to ../results/00-baseline-3CH-UNet-resnet18_kappa/20160816_L1A0001765570_2.npy


100%|██████████| 2/2 [00:07<00:00,  3.18s/it]


Saving to ../results/00-baseline-3CH-UNet-resnet18_kappa/20160827_L1A0001793003_2.npy


## 4.8 预测结果分析
### 4.8.1 全类别结果读取

In [11]:
ALL_y_pred, ALL_y_true = [],[]
P, R, ACC = [],[],[]
K, F1, F1_W, F1_micro, J, J_W, J_micro = [],[],[],[],[],[],[]
class_report_f1 = [ [] for i in range(16)]

for name in tqdm(SAVENAME):
    #name = SAVENAME[0]
    result_path = os.path.join(SAVEPATH, os.path.splitext(name)[0]+'_test.tif')
    label_path = os.path.join('../data/test_1106', os.path.splitext(name)[0]+'_label_clsindex.npy')  

    result = skimage.external.tifffile.imread(result_path)
    label = np.load(label_path)
    if result.shape[0] != label.shape[0] or result.shape[1] != label.shape[1]:
        print("错误！结果与标签尺寸不相等，结果为{}，标签为{}".format(result.shape, label.shape))

    result = get_cls_index(result)
    result = result.reshape(-1)
    label = label.reshape(-1)
    
    # 全部图片
    ALL_y_pred.append(result)
    ALL_y_true.append(label)
    
    # 调用计算metric的函数
    CAL = calculate(result, label, cm=False)
    P.append(CAL['P'])
    R.append(CAL['R'])
    ACC.append(CAL['ACC'])
    K.append(CAL['K'])
    F1.append(CAL['F1'])
    F1_W.append(CAL['F1-W'])
    F1_micro.append(CAL['F1-micro'])
    J.append(CAL['J'])
    J_W.append(CAL['J-W'])
    J_micro.append(CAL['J-micro'])
    for i in range(16):
        class_report_f1[i].append(CAL['CP']['{}'.format(i)]['f1-score'])

# 所有文件合并为一个
ALL_y_pred, ALL_y_true = np.array(ALL_y_pred).reshape(-1), np.array(ALL_y_true).reshape(-1)  
# 调用计算metric的函数
CAL = calculate(ALL_y_pred, ALL_y_true, cm=True)
P.append(CAL['P'])
R.append(CAL['R'])
ACC.append(CAL['ACC'])
K.append(CAL['K'])
F1.append(CAL['F1'])
F1_W.append(CAL['F1-W'])
F1_micro.append(CAL['F1-micro'])
J.append(CAL['J'])
J_W.append(CAL['J-W'])
J_micro.append(CAL['J-micro'])
for i in range(16):
    class_report_f1[i].append(CAL['CP']['{}'.format(i)]['f1-score'])
CM = CAL['CM']

100%|██████████| 10/10 [08:17<00:00, 49.34s/it]


### 4.8.2 五类别分析
### 4.8.3 文件保存

In [13]:
# 保存为相关文件
filename_list = [SAVENAME[i].split('.')[0] for i in range(10)]
filename_list.append('ALL')

dic = {'filename':filename_list, 
       'precision-micro':P, 'recall-micro':R, 'accuracy':ACC, 'kappa':K, 
       'F1-macro':F1, 'F1-weighted':F1_W, 'F1-micro':F1_micro,
       'jaccard-macro':J, 'jaccard-weighted':J_W, 'jaccard-micro':J_micro,
       'unknown':class_report_f1[0],
       'paddy field':class_report_f1[1],
       'irrigated land':class_report_f1[2],
       'dry land':class_report_f1[3],
       'garden land':class_report_f1[4],
       'arbor forest':class_report_f1[5],
       'shrub land':class_report_f1[6],
       'natural meadow':class_report_f1[7],
       'artificial meadow':class_report_f1[8],
       'industrial land':class_report_f1[9],
       'urban residential':class_report_f1[10],
       'rural residential':class_report_f1[11],
       'traffic land':class_report_f1[12],
       'river':class_report_f1[13],
       'lake':class_report_f1[14],
       'pond':class_report_f1[15]
      }

df = pd.DataFrame(dic)
df.to_csv(os.path.join(SAVEPATH, '{}_metrics_result.csv'.format(SaveName)), index=None)
np.save(os.path.join(SAVEPATH, 'confusion_matrix_{}.npy'.format(SaveName)),CM)
print("Kappa:{}, F1:{}, Jaccard:{}".format(CAL['K'], CAL['F1-W'], CAL['J-W']))

Kappa:0.6449582815238115, F1:0.7511957587337367, Jaccard:0.6179116639201255


### 4.8.4 混淆矩阵保存
* **混淆矩阵的图像生成需要使用py文件**

In [1]:
CM = np.load(os.path.join(SAVEPATH, 'confusion_matrix_{}.npy'.format(SaveName)))
# plot_confusion_matrix(CM, np.arange(16))
# plt.savefig(os.path.join(SAVEPATH, '{}_metrics_result.png'.format(SaveName)), format='png') 

NameError: name 'np' is not defined